# Introduction

In this notebook, I want to take a deep dive into sampling from a log likelihood with vanilla MCMC. This will help me build my intuition around Bayesian statistics and more importantly, how to design a probabilistic programming API.

## Recap Bayes' Rule

Bayes' rule is as follows:

$$P(H|D)= \frac{P(D|H)P(H)}{P(D)}$$

Here:

- $P(H)$ is the prior distribution on $H$ having not seen the data.
- $P(H|D)$ is the posterior belief on $H$ having seen the data $D$.
- $P(D|H)$ is the likelihood of the data.
- $P(D)$ is the normalizing constant, or the probability of the data.

## Translating $P(X)$s into code

I find I don't _truly_ understand something until I am able to translate it into code. Here, I will attempt to do so.

I will start with the components that I know. 

Firstly, I know how to calculate the likelihood of data given a hypothesis. This involves writing a function that takes in data points, and calculates the sum of log likelihoods (or product of likelihoods) assuming that the samples of data are i.i.d.

In [ ]:
import numpy as np
from scipy.stats import norm

In [ ]:
def norm_loglike(x, **norm_params):
    dist = norm(**norm_params)
    
    return np.sum(dist.logpdf(x))

Let's now use some fake data to test our understanding.

I will generate fake data from a $N(3,1)$ distribution, but calculate the log likelihood under a $N(0,1)$ distribution.

In [ ]:
xs = norm(loc=3, scale=1).rvs(100)

norm_loglike(xs, loc=0, scale=1)

Now, compare that with the log likelihood from the actual distribution.

In [ ]:
norm_loglike(xs, loc=3, scale=1)

Now, we still haven't had any "priors" injected here. Let's try it out with priors placed on the location of the Normal distribution, $m$.

Assume $m \sim N(0,10)$. In probability notation, 

$$P(m) = \frac{1}{\sigma_m \sqrt{2 \pi}} e^\frac{-(m-\mu_m)}{2 \sigma_m^2}$$

In [ ]:
def loglike_prior(m):
    dist = norm(loc=0, scale=10)
    return np.sum(dist.logpdf(m))

Let's test a few prior values, and calculate their log likelihood.

In [ ]:
print(loglike_prior(0))
print(loglike_prior(-1))
print(loglike_prior(1))
print(loglike_prior(-100))
print(loglike_prior(100))

Basically what we would expect. $0$ has the highest log likelihood, and log-likelihood goes down symmetrically.

Now, however, we need to use this prior in conjunction with the likelihood.

In [ ]:
def joint_log_prob(m, xs):
    return loglike_prior(m) + norm_loglike(xs, loc=m, scale=1)

When we multiply the prior probability distribution by the likelihood probability distribution, we are doing nothing more than summing up their log likelihoods.

In [ ]:
lls = []
for m in np.linspace(-10, 10, 100):
    ll = joint_log_prob(m, xs)
    lls.append(ll)

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.plot(np.linspace(-10, 10, 100), lls)

Let's find the max.

In [ ]:
np.linspace(-10, 10, 100)[lls.index(max(lls))]

Woohoo, this is close to the true value!

## Sampling Version

Let's now do it by _sampling_.

We are going to use the [Metropolis-Hastings algorithm](https://en.wikipedia.org/wiki/Metropolis%E2%80%93Hastings_algorithm).

Briefly, it works as follows:

- Initialize an arbitrary point.
- Choose density to propose new point (we will use $N(m_{t-1}, 1)$).
- For each iteration:
    - Generate candidate new candidate $m_t$.
    - Calculate acceptance ratio. We take advantage of the joint log probability L, by passing in the proposed value of $m_t$ into the logprob function, i.e. $L(m_t)$ and comparing it to $L(m_{t-1})$.
    - Now, we compute the ratio $r = \frac{L(m_t)}{L(m_{t-1})}$. 
    - Generate a new random number on the interval $p \sim U(0, 1)$.
    - Compare $p$ to $r$. 
        - If $p \leq r$, accept $m_t$.
        - If $p \gt r$, reject $m_t$ and continue sampling again with $m_{t-1}$.
        
Let's write it in code.

In [ ]:
# Metropolis-Hastings Sampling
m_prev = np.random.normal(0, 1)


history = dict()
ratio_history = dict()
for i in range(1000):
    history[i] = m_prev
    m_t = np.random.normal(m_prev, 5)
    
    # Compute negative joint log likelihood
    L_t = np.exp(joint_log_prob(m_t, xs))
    L_prev = np.exp(joint_log_prob(m_prev, xs))
    
    ratio = L_t / L_prev
    ratio_history[i] = L_t
    p = np.random.uniform(0, 1)
    if p <= ratio:
        m_prev = m_t
    

In [ ]:
np.array([i for i in history.values()]).mean()

It worked!

Now lies a challenge: How do we design an API that can handle not just `m` as the only variable, but arbitrary numbers of variables that have to be learned?

Let's try this out with a model that has two random variables instead of one. Here, we'll add variance of the likelihood to the list of RVs.

Here:

$$\mu \sim N(0, 10)$$
$$\sigma \sim Exp(2)$$
$$Y \sim N(\mu, \sigma)$$

In [ ]:
from scipy.stats import expon

def normal_prior_loglike(mu):
    return np.sum(norm(0, 10).logpdf(mu))

def sigma_prior_loglike(sigma):
    return np.sum(expon(scale=2).logpdf(sigma))

def data_loglike(mu, sigma, xs):
    return np.sum(norm(mu, sigma).logpdf(xs))

In [ ]:
def model_loglike(mu, sigma, xs):
    return normal_prior_loglike(mu) + sigma_prior_loglike(sigma) + data_loglike(mu, sigma, xs)

In [ ]:
# Metropolis-Hastings Sampling
mu_prev = np.random.normal(0, 1)
sigma_prev = np.random.normal(0, 1)

mu_history = dict()
sigma_history = dict()
ratio_history = dict()

# Let's test this with 100 steps.
for i in range(100):
    mu_history[i] = mu_prev
    sigma_history[i] = sigma_prev
    mu_t = np.random.normal(mu_prev, 5)
    sigma_t = np.random.normal(sigma_prev, 5)
    
    # Compute negative joint log likelihood
    LL_t = model_loglike(mu_t, sigma_t, xs)
    LL_prev = model_loglike(mu_prev, sigma_prev, xs)
    
    ratio = np.exp(LL_t - LL_prev)
    ratio_history[i] = ratio
    p = np.random.uniform(0, 1)
    if p <= ratio:
        m_prev = m_t

In [ ]:
ratio_history

Oh no, we get NaNs and infs in there! Why? Because we passed in invalid values into the logpdf of sigma. 

Sigma is bounded as a positive distribution, so we need to to somehow either:

1. Restrict sigma's proposal distribution to be positive, or
1. Transform sigma such that it lives on the real line, propose a new value on the real line, and then transform it back to original sigma.

Let's try the first strategy.

In [ ]:
# Metropolis-Hastings Sampling
mu_prev = np.random.normal(0, 1)
sigma_prev = np.random.exponential(scale=1)

mu_history = dict()
sigma_history = dict()
ratio_history = dict()

for i in range(10000):
    mu_history[i] = mu_prev
    sigma_history[i] = sigma_prev
    mu_t = np.random.normal(mu_prev, 0.1)
    sigma_t = abs(np.random.normal(sigma_prev, 0.1))
    
    # Compute negative joint log likelihood
    LL_t = model_loglike(mu_t, sigma_t, xs)
    LL_prev = model_loglike(mu_prev, sigma_prev, xs)
    
    
    # Calculate the ratio from the difference in log-likelihoods
    # (or a.k.a. ratio of likelihoods)
    diff_log_like = LL_t - LL_prev
    if diff_log_like > 0:
        ratio = 1
    else:
        ratio = np.exp(diff_log_like)
    if np.isinf(ratio) or np.isnan(ratio):
        raise ValueError(f"LL_t: {LL_t}, LL_prev: {LL_prev}")
                
    ratio_history[i] = ratio
    p = np.random.uniform(0, 1)
    
    if ratio >= p:
        mu_prev = mu_t
        sigma_prev = sigma_t

In [ ]:
plt.hist(sigma_history.values(), label="sigma")
plt.hist(mu_history.values(), label="mu")
plt.legend()

In [ ]:
np.array(list(sigma_history.values())).mean()

In [ ]:
np.array(list(mu_history.values())).mean()

Woohoo! It works!

Now, let's try the second option, where we propose a number in real space, then transform it into the space of the distribution support, and then evaluate the new proposed sample.

Doing so lets us use "standard" code to propose new distributions.

With a positive distribution ($X > 0$), a logical transform for the distribution is the log-transform, as it is simple, invertible, and won't have invalid values ($ln(0)$ is impossible here).

Because of this, the inverse of the transformation is going to be an exponential. Hence, we will sample in unconstrained, transformed space, use the inverse transform to go back to support space, and evaluate log likelihoods in support space.

In [ ]:
# Metropolis-Hastings Sampling
mu_prev = np.random.normal(0, 1)
sigma_prev = np.random.normal(0, 1)

mu_history = dict()
sigma_history = dict()
ratio_history = dict()

for i in range(1000):
    mu_t = np.random.normal(mu_prev, 0.1)
    # We'll make the changes here.
    # Firstly, we sample from the normal distribution.
    sigma_t = np.random.normal(sigma_prev, 0.1)
    # Then, we transform the distribution by its inverse transformation.
    # We will have to find a way to encapsulate this routine with a class method or function.
    sigma_t_tfm = np.exp(sigma_t)
    sigma_prev_tfm = np.exp(sigma_prev)

    # Record history
    mu_history[i] = mu_prev
    sigma_history[i] = sigma_prev_tfm

    # Compute negative joint log likelihood
    LL_t = model_loglike(mu_t, sigma_t_tfm, xs)
    LL_prev = model_loglike(mu_prev, sigma_prev_tfm, xs)

    # Calculate the ratio from the difference in log-likelihoods
    # (or a.k.a. ratio of likelihoods)
    diff_log_like = LL_t - LL_prev
    if diff_log_like > 0:
        ratio = 1
    else:
        ratio = np.exp(diff_log_like)
    if np.isinf(ratio) or np.isnan(ratio):
        raise ValueError(f"LL_t: {LL_t}, LL_prev: {LL_prev}")

    ratio_history[i] = ratio
    p = np.random.uniform(0, 1)

    if ratio >= p:
        mu_prev = mu_t
        sigma_prev = sigma_t

In [ ]:
plt.hist(list(sigma_history.values()), label="sigma")
plt.hist(list(mu_history.values()), label="mu")
plt.legend()

Notice the long tail of invalid values - that comes from the MCMC sampler stepping on its way to the region where the log likelihood is largest, but yet still isn't there. If we plot the histogram slightly differently, we will see what's happening.

In [ ]:
plt.plot(range(len(sigma_history)), list(sigma_history.values()))
plt.plot(range(len(mu_history)), list(mu_history.values()))

Indeed, there's a bit of a "burn-in" that the MCMC sampler needs before it reaches the region of "optimal" sampling. From manually observing the MCMC trace, we can see that sampling stabilizes around the correct values after 200+ steps. Let's set 200 to be the boundary at which we start plotting.

In [ ]:
plt.hist(list(sigma_history.values())[200:], label="sigma")
plt.hist(list(mu_history.values())[200:], label="mu")
plt.legend()

Looks much better now! Notice how we've also recovered back the correct values, with associated uncertainty modelled in as well!

## Designing an API for Sampling

Now, the code for Metropolis-Hastings sampling has been copied many times over at this point, so it's about time to design an API that can handle arbitrary numbers of samples and arbitrary numbers of scalar random variables.

Here are the design notes that we need to take care of.

### Generalized API

Firstly, the sampler API has to be consistent. A starter design might be to accept a model+data log-likelihood and its associated parameters, though thinking further about it, one might want to instead accept a "container" of sorts that wraps everything together (i.e. the data + model parameters + joint log likelihood).

### Further Considerations

Distributions need to have automatic transformations enabled.
We need to be able to pass in a value in unconstrained, transformed space, but evaluate the log-likelihood in constrained, support space (wherever applicable). 

In PyMC3, we have context managers that handle things, but maybe a simpler way of handling this is to wrap distributions in a Python object, and provide the appropriate class methods, perhaps as follows:

```python
class Exponential(PositiveDistribution):
    def sumlogpdf(x):
        return np.sum(self.dist.logpdf(x))
    
    def sumlogpdf_tfm(x):
        x = self.inverse_transform(x)
        return self.logpdf(x)
```


In doing so, we could possibly avoid needing to explicitly specify proposal distributions and their transformations in the sampling loop.

In [ ]:
class Distribution(object):
    def __init__(self):
        return NotImplementedError("Do not use abstract distribution class!")

    def transform(self, x):
        """Invertible transform that converts real number to distribution support."""
        return x
    
    def inv_transform(self, x):
        """Inverse of transform."""
        return x
    
    def sumlogpdf_tfm(self, x):
        x = self.inv_transform(x)
        return self.sumlogpdf(x)
    
    def sumlogpdf(self, x):
        return np.sum(self.dist.logpdf(x))
    
    def sample(self, n):
        return self.dist.rvs(n)

    
class PositiveDistribution(Distribution):
    def transform(self, x):
        """Transformation for positive distributions is a log-transform."""
        return np.log(x)

    def inv_transform(self, x):
        return np.exp(x)


class Normal(Distribution):
    def __init__(self, mu, sigma):
        self.mu = mu
        self.sigma = sigma
        self.dist = norm(loc=mu, scale=sigma)


class Exponential(PositiveDistribution):
    def __init__(self, lam):
        self.lam = lam
        self.dist = expon(scale=lam)

Now, let's run some simple sanity checks to make sure everything was implemented correctly.

Firstly, we are going to calculate the log_prob of data for the exponential distribution.

In [ ]:
x = np.random.exponential(scale=1.0, size=(10))

X = Exponential(lam=1)
X.sumlogpdf(x)

Now, let's try generating data from a normal distribution, and call on the transformed sumlogpdf.

In [ ]:
x = np.random.normal(loc=0.0, scale=1.0, size=(10,))
X = Exponential(lam=1)
X.sumlogpdf_tfm(x)

Now, let's try generating data from a normal distribution, and call on the transformed and non-transformed sumlogpdf of the Normal distribution. The results should be identical.

In [ ]:
x = np.random.normal(loc=0, scale=1, size=10)
X = Normal(mu=0, sigma=1)
X.sumlogpdf(x), X.sumlogpdf_tfm(x)

It looks like we're close to finishing the components needed in the distributions library of a PPL.

Another component we might need to worry about is that when we "pass" a distribution into another distribution as a parameter, what we are really doing is sampling a value from that distribution and then passing it into the next.

As an example the following model in equations:

$$\mu \sim N(0, 1)$$

$$L \sim N(\mu, 1)$$

translates to the following code (while simulating data from a model):

```python
mu = Normal(0, 1)
L = Normal(mu.sample(1), 1)
```

However, while in inference mode, that is, when trying to sample posterior values of `mu`, we instead need the above code to be transformed into the following pseudocode:

```python
loglike_mu(mu) + loglike_L(mu, L, data)
```


TO BE CONTINUED...

Now, we're going to see whether we can use this to build a model that can return a log-probability function. That is basically what a model has to do.

But first, some ideas to chew on.

**Firstly**, the model has to know what its random variables, likelihood, and data are. It's possible to have more than one random variable, more than one likelihood, and hence more than one data source passed in. Knowing this implies that there is "state" involved, and hence a class-based implementation will make things easier to track than a function-based implementation.

**Secondly**, the model has to know which random variables are passed into the likelihood function. Only then we can construct the evaluation of the likelihood correctly. To remind ourselves on why:

```python
def model_loglike(mu, sigma, xs):
    return normal_prior_loglike(mu) + sigma_prior_loglike(sigma) + data_loglike(mu, sigma, xs)
```

Note here how `mu` has to be passed into the `normal_prior_loglike` and the `data_loglike`, while `sigma` has to be passed into the `sigma_prior_loglike` and the `data_loglike`. This induces a "graph" of sorts, which reveals to us another concept: Bayesian models have a "graphical" representation, through which data and model parameters flow through from the priors to the likelihood.

**Thirdly**, for diagnostic purposes, we need to be able to do a "prior sampling" step, that is to say, simulate what data would look like if generated from our priors. In the absence of data, this step lets us calibrate our priors better: we could impose soft constraints (e.g. by tuning variance parameters) to ensure that the range of our data fall within the right order of mangitude(s) before fitting. We thus need to have an API that has something like `model.prior_samples()`, which then returns distributions for each of the latent parameters and simulated data from the model likelihood.

**Fourthly**, again for diagnostic purposes, we need to be able to do a "posterior sampling" step, that is to simulate what the data looks like under the posterior sampled parameters. The steps involved here are to take each of the sampled posterior values of the random variables, pass them to the appropriate parts of the log-likelihood function, and generate data from the log-likelihood.

**Fifthly**, we need the ability to do 

We need to decompose this into its constituent components.

1. We have a `sampler` to sample from the joint log-likelihood, which should accept proposed random variable values and data, and return a scalar.
2. We have a `model` specification, which should accept data and return a log-likelihood function and the random variables values to evaluate.

An API sketch might look like the following: